go to   Runtime > Change runtime type >  Hardware accelerator.
choose GPU to make the model run faster

# Mounting Gdrive and loading modules (Change depending on your configuration)

In [13]:
import numpy as np
# Mounting gdrive
from google.colab import drive
drive.mount('/gdrive')

# Installing missing modules
!pip install mne

# Importing coleeg functions (reload function is used to successfully update any modification in coleeg.py file)
import sys
from importlib import reload
sys.path.append("/gdrive/MyDrive/coleeg/ver2/")
import coleeg
from coleeg import *

# disable line breaking when printing array objects
np.set_printoptions(linewidth=np.nan)


# creating folder for results
import os.path
from os import path
if path.exists('/gdrive/MyDrive/results') == False:
  os.mkdir('/gdrive/MyDrive/results')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [14]:
# reload coleeg  (run it when coleeg.py is changed)
reload(coleeg)
from coleeg import *

# Show info about coleeg

In [15]:
print(coleeg.__version__)

2.0


In [16]:
coleeg_info()


* Coleeg is an open source initiative for collaborating on making a piece of software for investigating the use of Neural Networks in EEG signal classification on different datasets.

* License GPL V2.0

## Team:
Mahmoud Alnaanah (malnaanah@gmail.com)
Moutz Wahdow (m.wahdow@gmail.com)

## Team:

Mahmoud Alnaanah (malnaanah@gmail.com)
Moutz Wahdow (m.wahdow@gmail.com)

## How to install Coleeg to your google drive

  1- Make a new directory called "coleeg" on your google drive
  2- Copy Coleeg version folder, i.e. "ver2", inside the folder "coleeg'

## How to install dataset files on your Google drive

  * Physionet
    1- Go to the link: https://physionet.org/static/published-projects/eegmmidb/eeg-motor-movementimagery-dataset-1.0.0.zip
    2- Upload dataset file to your google drive inside the folder "datasets"

  * BCI competetion IV-2a  
    1- Got to the link: http://www.bbci.de/competition/iv/#download
    2- Enter your name and email address and press "I Agree"
    3- Check your

# Obtaining data files (Change depending on your configuration.)

In [ ]:
#==================== Options =======================
dataset = 'bcicomptIV2a' # Select one of 'physionet', 'ttk', 'bcicomptIV2a'
#=====================================================

start_time = time.time()
unzip_dataset(dataset=dataset)
print(f'Time = {timedelta(seconds=round(time.time() - start_time))}')


Unzipping data.
Unzipping done.
Time = 0:00:12


# Loading data

In [ ]:
#====================== Options  ==============================
# Determining dataset
dataset = 'bcicomptIV2a' # Select one of 'physionet', 'ttk', 'bcicomptIV2a'

resample_freq = 100 # resample frequency in Hz (Set to None to deactivate)

Bands = None 
Bands = [(0.5, 8.0), (8.0, 13.0), (13.0, 40.0)] # Uncomment this line to use filters
#Bands = [(0.5, 49.5)] # Uncomment this line to use filters

notch_freqs = None # frequency (or list of frequencies) to which a notch filter is applied
tmin = 0.0
tmax = 2.0

# select tasks for Physionet dataset
# Tasks = np.array([[3,7,11],[5,9,13]]) # Real movement
Tasks=np.array([[4,8,12],[6,10,14]]) # Uncommnet this line for imagined movement
#=============================================================================================
# set sample frequency
Sample_Freqs = {'physionet':160, 'ttk':500, 'bcicomptIV2a':250}
if resample_freq is not None:
  Fs = resample_freq
else:
  Fs = Sample_Freqs[dataset]

# Delete variables
data_input,data_x, data_x2D, data_y, data_yc,  train_x, train_x2D, train_y, train_yc, test_x, test_x2D, test_y, test_yc = 0,0,0,0,0,0,0,0,0,0,0,0,0
del data_input, data_x, data_x2D, data_y, data_yc,  train_x, train_x2D, train_y, train_yc, test_x, test_x2D, test_y, test_yc

start_time = time.time()

# loading data
if dataset is 'bcicomptIV2a':
  import warnings
  warnings.filterwarnings("ignore", category=DeprecationWarning) # disable annoying error messages from python
  data_x, data_y, data_index = get_data_bcicomptIV2a(resample_freq=resample_freq, Bands=Bands, tmin=tmin, tmax=(tmax-1/Fs), Baseline=None)
elif dataset is 'physionet':
  data_x, data_y, data_index = get_data_physionet(resample_freq=resample_freq, Tasks=Tasks, Bands=Bands,  tmin=tmin, tmax=(tmax-1/Fs), Baseline=None, notch_freqs=notch_freqs)
  #data_x, data_y, data_index = get_data_physionet(Subjects=np.arange(1,101),data_path=data_path, resample_freq=resample_freq, Tasks=Tasks, Bands=Bands,  tmin=tmin, tmax=(tmax-1/Fs), Baseline=None, notch_freqs=notch_freqs)
elif dataset is 'ttk':
  data_x, data_y, data_index = get_data_ttk(resample_freq=resample_freq, Bands=Bands, tmin=tmin, tmax=(tmax-1/Fs), Baseline=None)
else:
  print('Wrong dataset')

print(f'\nTime = {timedelta(seconds=round(time.time() - start_time))}')

# balancing data to minimum occured class. Balancing is done for each subject
data_x, data_y, data_index = balance(data_x, data_y, data_index)

# normalizing data (mean => 0, standard deviation => 1)
normalize(data_x)


No subject excluded
Loading subject 18/18
Time = 0:01:18
Balancing data done.
Normalizing data done.


# Model Evaluation

In [ ]:
#====================== Options ===========================
# model_list: dictionary for models and list of epochs for training, modify according to your configuration
# Select one or more of the following models: 'Basic','CNN1D', 'EEGNet', 'ShallowConvNet', 'DeepConvNet', 'CNN2D', 'CNN3D', 'TimeDist', 'CNN1D_MFBF', 'EEGNet_fusion'
# model_list = ['CNN1D','EEGNet_fusion', 'EEGNet', 'ShallowConvNet', 'DeepConvNet' ]
model_list = ['CNN1D_MFBF']

splits = 5 # number of splits to divide data into, testing ratio = 1/splits
#splits = data_index.shape[0] # uncomment for per-subject evaluations (align_to_subject should be True)
Iterations = range(splits) # splits to include in validation, for kfold validation Iterations = range(splits)
#Iterations = [0] # uncomment to set iterations manually, helpful to do single iteration when validation takes long time and colab disconnect before finishing all iterations.

epochs = 500 # number of epochs

align_to_subject = True # (True/False) align split boundary to subjects (necessery for cross validation)
                        # number of subjects in each split may vary.
shuffle = True # shuffle data before splitting, has no effect if align_to_subjects is true
batch_norm = True# Enable/Disable Batch normalization in the models: 'EEGNet', 'ShallowConvNet', 'DeepConvNet'
show_summary = False # Show summery of the model
verbose = True # show progress for each epoch
dropout_rate=0.5 # dropout rate for the models: CNN1D, CNN2D, CNN3D, TimeDist, CNN1D_MFBF
play_audio = False # play audio when evaluation finishes
#===========================================================


if 'model' in locals():
  del model

######################### Function Start ####################################
model, results, time_results,  subject_results = evaluate_model(
    model_list,dataset,Bands, data_x, data_y, data_index,splits,Iterations,
    epochs, batch_size=64, verbose=verbose, show_summary = show_summary, batch_norm=batch_norm,
    dropout_rate=dropout_rate, align_to_subject=align_to_subject, shuffle=shuffle, play_audio=play_audio)





In [ ]:
# plotting history data
plot_results(results)

In [ ]:
# Prediction for each class
# ======================= options =======================
model_type = 'EEGNet_fusion'
splits = 5 # number of splits to divide data into, testing ratio = 1/splits
Iterations = range(splits) # splits to include in testing
#Iterations = [0,1,2,..] # uncomment to set iterations manually, index start from 0
align_to_subject = True
shuffle = True # shuffle data before splitting, has no effect if align_to_subjects is true
# ========================================================

predict_model(model,model_type, data_x, data_y,data_index,splits,Iterations, align_to_subject=align_to_subject, shuffle=shuffle)

# Removing classes

In [ ]:
classes = None
# classes = [0,3,4] # Uncomment and put the classes to be removed

if classes is not None:
  data_y, data_x, data_index  = remove_classes(data_y, classes, data_x, data_index = data_index)
  data_yc = to_categorical(data_y)

# Visualization (video array)

In [ ]:
# Generating 2D mapped data
pos_map = get_pos_map(dataset) # positions for 2D map conversion
data_x2D = make_into_2d(data_x,pos_map)

In [ ]:
# Generating video array for random samples
video_array(data_x2D, data_y, Class=1, Band=0, Rows=2, Cols=3)

samples = [3397 3450  404  919 1373 4465 1977 4190 3223 2481 4027 1668]
